1.加载训练的data pipeline 但不训练

In [20]:
import torch
from util.Trainer import Trainer

from util.dataloader import ChemDataRaw, MyTokenizer, get_data_loader, ChemDataSet
from util.TrainInits import init_seed, print_model_parameters
import configparser
from torch import nn 
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
init_seed(114514)

# config
config = configparser.ConfigParser()
config.read("config-small.conf")
vocab_file = config['data']['vocab_file']
from_exist = bool(eval(config['train']['from_exist']))


device = config['model']['device']


In [21]:

# dataset
tokenizer = MyTokenizer(vocab_file)
config['data']['vocab_size'] = str(tokenizer.get_vocab_size())
mol_files = config['data']['mol_files'].split(',')
react_files = config['data']['react_files'].split(',')
retro_files = config['data']['retro_files'].split(',')
chemdata = ChemDataRaw(mol_files, react_files, retro_files)


12788it [00:00, 127867.01it/s]

100001it [00:00, 203013.78it/s]


processing /home/data/wd/correct_norm_reaction.txt


4883729it [00:45, 106774.64it/s]


processing /home/data/wd/correct_norm_retro_no_ca.txt


1801113it [00:23, 75739.84it/s] 


100001 4883729 1801113
10000 4883729 1801113
-------------- showing examples --------------------------------
molecules:
[2.023179999999999, 59.37, 0.8235513388578103, 2.2812516757370815] O=C(CCc1ccccc1)NCCCN1CCNCC1.CN1CCN(CCC(C)(C)NC(=O)CCc2ccc(C#N)cc2)CC1!
reactions:
[1.45928, 157.45, 0.5691895676039367, 2.5280768445981483] CCCCOC(=O)c1nc(C#N)c2ccc(S(=O)(=O)c3ccccc3)cc2c1O.NCC(=O)O>>O=C(c1nc(C#N)c2ccc(S(=O)(=O)c3ccccc3)cc2c1O)NCC(=O)O!
retro:
[4.994900000000005, 60.440000000000005, 0.4500855369554003, 4.574026652089674] CC(=O)OC(C)=O.F[C@H]1[C@H]2[C@@H]3CCC(=O)C=C3C[C@@H](CCCCCO)[C@@H]2[C@H]2CCC(=O)[C@@]2(C)C1<<F[C@H]1[C@H]2[C@@H]3CCC(=O)C=C3C[C@@H](CCCCCOC(C)=O)[C@@H]2[C@H]2CCC(=O)[C@@]2(C)C1!
----------------------------------------------------------------


In [22]:

# train_set = ChemDataSet(chemdata.train_data, tokenizer, int(config['model']['n_positions']))
# props, train_str, attention_mask = train_set[0]
# print(props.shape, train_str.shape, attention_mask.shape)
# props, train_str, attention_mask = train_set[1]
# print(props.shape, train_str.shape, attention_mask.shape)
# exit()


train_loader, val_loader, test_loader = get_data_loader(chemdata, 
                                                        int(config['train']['batch_size']), 
                                                        tokenizer,
                                                        int(config['model']['n_positions'])-1)


In [37]:
from util.gpt2_copy import generate_model
if from_exist:
    model = torch.load(os.path.join(config['train']['log_dir'], 'best_model.pth'))
else:
    model = generate_model(config)
model = model.to(device)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    # else:
    #     nn.init.uniform_(p)
# print_model_parameters(model, only_num=False)


In [38]:
for batch_idx, (props, seqs, attention_mask) in enumerate(train_loader):
    props = props.to(device)
    seqs = seqs.to(device)
    attention_mask = attention_mask.to(device)
    labels = [seqs, props]
    data = [props, seqs[..., :-1]]

    output = model(data, labels, attention_mask)
    # print(output)
    break

In [39]:
eos_token = '!'
eos_ids = tokenizer.char2num[eos_token]

In [40]:
pad_token = '!'


encoder_input就在batch里面


In [41]:
input_ids = seqs[0].unsqueeze(0)

In [42]:
input_ids.shape

torch.Size([1, 1024])

In [43]:
attn_mask = attention_mask[0].unsqueeze(0)

In [44]:
attn_mask.shape

torch.Size([1, 1024])

In [45]:
attn_mask = attn_mask[attn_mask == 1]
input_ids = input_ids[input_ids != 68]

In [46]:
attn_mask

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.], device='cuda:3')

input_ids 和 attn_mask就是输出，统一成encoded_input字典

In [47]:
encoded_input={}
encoded_input['input_ids'] = input_ids
encoded_input['attention_mask'] = attn_mask

In [48]:
encoded_input

{'input_ids': tensor([58, 46, 58, 39, 23,  8, 23, 23, 39, 58, 59, 23, 39, 26, 58, 59, 23, 39,
         58, 59, 23,  8, 59, 49, 26, 35, 51, 58, 39, 58, 39, 49, 26, 59, 26, 58,
         58, 59, 58, 39, 49, 26, 59, 26, 58, 58, 34, 34, 51, 39, 58, 39, 58, 39,
         49, 26, 59, 26, 58, 58, 59, 58, 39, 49, 26, 59, 26, 58, 58, 59, 58, 39,
         23,  8, 23, 23, 39, 58, 59, 23, 39, 26, 58, 59, 23, 39, 58, 59, 23,  8,
         59, 49, 26], device='cuda:3'),
 'attention_mask': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.], device='cuda:3')}

In [49]:
generation_configs = {
    'beam_size': 1, # test greedy search
    'max_gen_len': 50,
    'end_ids': 68,
    'pad_id': 68,
    'no_repeat_ngram_size': 1,
    'do_sample': True,
    'temperature': 0.8,
    'top_k': 10,
    'top_p': 0.8,
}

In [52]:
model.generate()

TypeError: The current model class (ChemGPT) is not compatible with `.generate()`, as it doesn't have a language model head.

In [51]:
model

ChemGPT(
  (transformer): GPT2Model(
    (wte): Embedding(71, 384)
    (wpe): Embedding(1024, 384)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (prop_in_linear): Linear(in_features=4, out_features=384, bias=True)
  (prop_out_linear): Linear(in_features=384, out_features=4, bias=True)
  (lm_head): Linear(in_features=384, out_features=71, bias=

In [1]:
mol_files = r'/home/data/wd/zinc_data.txt'

with open('/home/data/wd/zinc_data.txt', 'r') as file:
    lines = file.readlines()
    first_few_lines = lines[:5]  # Change the number 5 to the desired number of lines
    print(first_few_lines)


['N/C(=N\\OC(=O)c1ccccc1Cl)c1nccnc1Oc1cccc(C(F)(F)F)c1\n', 'COc1ccc(CCN2Cc3c(N=Cc4ccc(O)cc4O)cccc3C2=O)cc1OC\n', 'C/C(=N\\OCC(=O)O/N=C(\\N)COc1ccc(Cl)cc1Cl)c1ccc(Cl)cc1\n', 'CCCCCCCC/C=C\\CCCCCCCC(=O)OC[C@H](O)COC[C@H](O)CO\n', 'O=S(OCC(F)(F)C(F)(F)C(F)(F)F)OCC(F)(F)C(F)(F)C(F)(F)F\n']


In [4]:
import pandas as pd

df = pd.read_csv('/home/Zhouyu/MODEL/code/zinc250k.csv', usecols=['smiles'])


In [6]:
df.to_csv('/home/Zhouyu/MODEL/code/zinc250k.txt', index=False, header=False)

In [7]:
with open('/home/Zhouyu/MODEL/code/zinc250k.txt', 'r') as file:
    lines = file.readlines()
    first_few_lines = lines[:5]  # Change the number 5 to the desired number of lines
    print(first_few_lines)

['CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n', 'C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n', 'N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1\n', 'CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCCC3)C1\n', 'N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#N)C12CCCCC2\n']
